Variational AutoEncoder (VAE) with CelebA
=========================================
---
Formation Introduction au Deep Learning  (FIDLE) - S. Arias, E. Maldonado, JL. Parouty - CNRS/SARI/DEVLOG - 2020  

## Episode 1 - Train a model

 - Defining a VAE model
 - Build the model
 - Train it
 - Follow the learning process with Tensorboard


## Step 1 - Setup environment
### 1.1 - Python stuff

In [1]:
import tensorflow as tf
import numpy as np
import os,sys
from importlib import reload

import modules.vae
import modules.data_generator

reload(modules.data_generator)
reload(modules.vae)

from modules.vae  import VariationalAutoencoder
from modules.data_generator import DataGenerator

sys.path.append('..')
import fidle.pwk as ooo
reload(ooo)

ooo.init()

VariationalAutoencoder.about()
DataGenerator.about()


FIDLE 2020 - Practical Work Module
Version              : 0.2.8
Run time             : Friday 14 February 2020, 00:07:28
TensorFlow version   : 2.0.0
Keras version        : 2.2.4-tf

FIDLE 2020 - Variational AutoEncoder (VAE)
TensorFlow version   : 2.0.0
VAE version          : 1.28

FIDLE 2020 - DataGenerator
Version              : 0.4.1


### 1.2 - The good place

In [2]:
place, dataset_dir = ooo.good_place( { 'GRICAD' : f'{os.getenv("SCRATCH_DIR","")}/PROJECTS/pr-fidle/datasets/celeba',
                                       'IDRIS'  : f'{os.getenv("WORK","")}/datasets/celeba'    } )

# ---- train/test datasets

train_dir    = f'{dataset_dir}/clusters-M.train'
test_dir     = f'{dataset_dir}/clusters-M.test'

Well, we should be at IDRIS !
We are going to use: /gpfswork/rech/mlh/uja62cb/datasets/celeba


## Step 2 - DataGenerator and validation data
Ok, everything's perfect, now let's instantiate our generator for the entire dataset.

In [3]:
data_gen = DataGenerator(train_dir, 32, k_size=1)
x_test   = np.load(f'{test_dir}/images-000.npy')

print(f'Data generator : {len(data_gen)} batchs of {data_gen.batch_size} images, or {data_gen.dataset_size} images')
print(f'x_test : {len(x_test)} images')

Data generator : 6250 batchs of 32 images, or 200000 images
x_test : 2599 images


## Step 3 - Get VAE model

In [4]:
tag = f'CelebA.052-M.{os.getenv("SLURM_JOB_ID","unknown")}'

input_shape = (192, 160, 3)
z_dim       = 200
verbose     = 0

encoder= [ {'type':'Conv2D',          'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
         ]

decoder= [ {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':3,  'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'sigmoid'}
         ]

vae = modules.vae.VariationalAutoencoder(input_shape    = input_shape, 
                                         encoder_layers = encoder, 
                                         decoder_layers = decoder,
                                         z_dim          = z_dim, 
                                         verbose        = verbose,
                                         run_tag        = tag)
vae.save(model=None)

Model initialized.
Outputs will be in  : ./run/CelebA.052-M.810156
Config saved in     : ./run/CelebA.052-M.810156/models/vae_config.json


## Step 4 - Compile it

In [5]:
optimizer     = tf.keras.optimizers.Adam(1e-4)
r_loss_factor = 10000

vae.compile(optimizer, r_loss_factor)

Compiled.


## Step 5 - Train
For 10 epochs, adam optimizer :  
- Run time at IDRIS : 1299.77 sec. - 0:21:39
- Run time at GRICAD : 2092.77 sec. - 0:34:52

In [6]:
epochs            = 20
initial_epoch     = 0

In [7]:
vae.train(data_generator    = data_gen,
          x_test            = x_test,
          epochs            = epochs,
          initial_epoch     = initial_epoch
         )

Epoch 1/20
6250/6250 [==============================] - 342s 55ms/step - loss: 332.4792 - vae_r_loss: 282.9655 - vae_kl_loss: 49.5134 - val_loss: 235.4288 - val_vae_r_loss: 187.4334 - val_vae_kl_loss: 48.1192
Epoch 2/20
6250/6250 [==============================] - 337s 54ms/step - loss: 224.0962 - vae_r_loss: 166.4602 - vae_kl_loss: 57.6360 - val_loss: 210.9632 - val_vae_r_loss: 155.0925 - val_vae_kl_loss: 56.0114
Epoch 4/20
6250/6250 [==============================] - 333s 53ms/step - loss: 214.5463 - vae_r_loss: 155.7666 - vae_kl_loss: 58.7794 - val_loss: 203.8241 - val_vae_r_loss: 147.3248 - val_vae_kl_loss: 56.5778
Epoch 7/20
6250/6250 [==============================] - 327s 52ms/step - loss: 211.1459 - vae_r_loss: 152.4026 - vae_kl_loss: 58.7437 - val_loss: 201.1862 - val_vae_r_loss: 145.6906 - val_vae_kl_loss: 55.6326
Epoch 10/20
6250/6250 [==============================] - 335s 54ms/step - loss: 209.7628 - vae_r_loss: 151.0874 - vae_kl_loss: 58.6756 - val_loss: 202.3954 - val_va

----
That's all folks !